In [17]:
import yfinance as yf
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from sklearn.preprocessing import MinMaxScaler

from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.models import Sequential

### **Download Market Data**

In [23]:


num_of_years_back = 10
end_date = datetime.today()
start_date = end_date - timedelta(days=365 * num_of_years_back)
ticker_symbol  = 'GOOG'
data = yf.download(ticker_symbol , start=start_date.strftime('%Y-%m-%d'), end=end_date.strftime('%Y-%m-%d'))

data.reset_index(inplace = True) # Set index instead of dates to range from 0 to 1. 
data.dropna(inplace = True)

[*********************100%%**********************]  1 of 1 completed


### **Split - Scale Data**

In [33]:
# Splt
data_train = pd.DataFrame(data.Close[0:int(len(data) * 0.8)])
data_test = pd.DataFrame(data.Close[int(len(data) * 0.8): len(data)])

train_len = data_train.shape[0]
test_len = data_test.shape[0]

# Scale Data 
scaler = MinMaxScaler(feature_range = (0,1))
data_train_scale = scaler.fit_transform(data_train)

In [31]:
x = []
y = []
for i in range(100, data_train_scale.shape[0]):
    x.append(data_train_scale[i-100:i])
    y.append(data_train_scale[i,0])

x, y = np.array(x), np.array(y)

### **Create a LSTM Model**

In [39]:
## Model Creation
model = Sequential()

model.add(LSTM(units = 50, activation = 'relu', return_sequences = True, #return_sequences = output of one layer becomes the input of the next one
               input_shape = ((x.shape[1],1))))
model.add(Dropout(0.2))

model.add(LSTM(units = 60, activation = 'relu', return_sequences = True))
model.add(Dropout(0.3))

model.add(LSTM(units = 80, activation = 'relu', return_sequences = True))
model.add(Dropout(0.4))

model.add(LSTM(units = 120, activation = 'relu'))
model.add(Dropout(0.5))

model.add(Dense(units = 1))

## Model Compile
model.compile(optimizer = 'adam', loss = 'mean_squared_error')
model.fit(x , y , epochs = 50, batch_size = 32, verbose = 1)

c:\Users\Altair\AppData\Local\Programs\Python\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 26s 225ms/step - loss: 0.0752
Epoch 2/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 13s 213ms/step - loss: 0.0073
Epoch 3/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 20s 199ms/step - loss: 0.0067
Epoch 4/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 13s 223ms/step - loss: 0.0049
Epoch 5/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 21s 222ms/step - loss: 0.0048
Epoch 6/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 25s 302ms/step - loss: 0.0066
Epoch 7/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 18s 305ms/step - loss: 0.0046
Epoch 8/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 19s 272ms/step - loss: 0.0042
Epoch 9/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 21s 270ms/step - loss: 0.0050
Epoch 10/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 22s 297ms/step - loss: 0.0041
Epoch 11/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 20s 287ms/step - loss: 0.0036
Epoch 12/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 19s 260ms/step - loss: 0.0033
Epoch 13/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 24s 319ms/step - loss: 0.0039
Epoch 14/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 17s 254ms/step - loss: 0.0037
Epoch 15/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 17s 

In [40]:
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_4 (LSTM)                   │ (None, 100, 50)        │        10,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 100, 50)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_5 (LSTM)                   │ (None, 100, 60)        │        26,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 100, 60)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_6 (LSTM)                   │ (None, 100, 80)        │        45,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 100, 80)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_7 (LSTM)                   │ (None, 120)            │        96,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 120)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           121 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 536,285 (2.05 MB)

 Trainable params: 178,761 (698.29 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 357,524 (1.36 MB)

### **Model Testing**

In [44]:
past_100_days = data_train.tail(100)
data_test_plus = pd.concat([past_100_days, data_test], ignore_index = True)

data_test_scale = scaler.fit_transform(data_test_plus)

x = []
y = []
for i in range(100, data_test_scale.shape[0]):
    x.append(data_test_scale[i-100:i])
    y.append(data_test_scale[i,0])

x, y = np.array(x), np.array(y)

In [46]:
# Predict
y_predict = model.predict(x)
scaling_factor = 1/scaler.scale_
y_predict = y_predict * scaling_factor
y_real = y * scaling_factor 

16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 83ms/step


In [50]:
# Save model 
model.save('Stock Predictions Model.keras')